In [1]:
%pip install requests
%pip install bs4
%pip install flask


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from requests import get
from bs4 import BeautifulSoup
from flask import Flask, render_template, request, redirect, send_file


In [3]:
def save_to_file(file_name, jobs):
    file = open(f"{file_name}.csv", "w")
    file.write("Position,Company,URL\n")

    for job in jobs:
        file.write(f"{job['title']}," + f"{job['company']}," + f"{job['link']}\n")
    file.close()


In [4]:
def remoteok_extract_jobs(keyword):
    results = []
    url = f"https://remoteok.com/remote-{keyword}-jobs"
    request = get(url, headers={"User-Agent": "Kimchi"})
    if request.status_code == 200:
        soup = BeautifulSoup(request.text, "html.parser")
        jobs = soup.find_all("tr", {"class": "job"})
        for job in jobs:
            # ! strip()은 공백을 제거해준다.
            title = job.find("h2", {"itemprop": "title"}).text.strip()
            company = job.find("h3", {"itemprop": "name"}).text.strip()
            link = job.find("a", {"itemprop": "url"})["href"]
            job_data = {
                "title": title,
                "company": company,
                "link": f"https://remoteok.com{link}"
            }
            results.append(job_data)
            print(f"{title} - {company} - {link}")
            # write your ✨magical✨ code here
        return results
    else:
        print("Can't get jobs.")


In [5]:
def weworkremotely_extract_jobs(keyword):
    results = []
    url = f"https://weworkremotely.com/remote-jobs/search?term={keyword}"
    request = get(url, headers={"User-Agent": "Kimchi"})
    if request.status_code == 200:
        soup = BeautifulSoup(request.text, "html.parser")
        jobs = soup.find_all("li", {"class": "feature"})
        for job in jobs:
            title = job.find("span", {"class": "title"}).text.strip()
            company = job.find("span", {"class": "company"}).text.strip()
            link = job.find_all("a")[1]['href']
            
            job_data = {
                "title": title, 
                "company": company,
                "link": f"https://weworkremotely.com{link}"
            }
            
            results.append(job_data)
            print(f"{title} - {company} - {link}")
        return results


In [6]:
app = Flask("JobScrapper")


@app.route("/")
def home():
    return render_template("home.html", name="jeongbin")


@app.route("/hello")
def hello():
    return 'hello!'


@app.route("/search")
def search():
    keyword = request.args.get("keyword")
    if keyword:
        keyword = keyword.lower()
        remoteok = remoteok_extract_jobs(keyword)
        weworkremotely = weworkremotely_extract_jobs(keyword)
        jobs = remoteok + weworkremotely
        
        print(jobs)
        return render_template("search.html", keyword=keyword, jobs=jobs)
    else:
        return redirect("/")
    

@app.route("/export")
def export():
    keyword = request.args.get("keyword")
    if keyword:
        keyword = keyword.lower()
        jobs = remoteok_extract_jobs(keyword) + weworkremotely_extract_jobs(keyword)
        save_to_file(keyword, jobs)
    return send_file(f"{keyword}.csv", as_attachment=True)


In [7]:
app.run(host="0.0.0.0", port=8000)

 * Serving Flask app 'JobScrapper'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.0.194:8000
Press CTRL+C to quit


AI engineer - Sticker Mule - /remote-jobs/remote-ai-engineer-sticker-mule-213158
Fullstack developer senior - CloudDevs - /remote-jobs/remote-fullstack-developer-senior-clouddevs-207318
Senior Backend Python Developer - Proxify - /remote-jobs/remote-senior-backend-python-developer-proxify-206355
Workflow Automation Specialist - Opzer.co - /remote-jobs/remote-workflow-automation-specialist-opzer-co-202539
Data Engineer - Art Blocks - /remote-jobs/remote-data-engineer-art-blocks-201880
Data Engineer Intern - DeFiner - /remote-jobs/remote-data-engineer-intern-definer-199886
DevOps Engineer Freelance - Braintrust - /remote-jobs/remote-devops-engineer-freelance-braintrust-199343
Data Engineer - Scalapay - /remote-jobs/remote-data-engineer-scalapay-188410
Engineer Golang Python - FinCompare - /remote-jobs/remote-engineer-golang-python-fincompare-183780
Senior DevOps Engineer - Twist Bioscience - /remote-jobs/remote-senior-devops-engineer-twist-bioscience-167248
Senior Software Engineer Test 

127.0.0.1 - - [25/Mar/2023 23:00:39] "GET /search?keyword=python HTTP/1.1" 200 -


Senior Python Engineer: Long-term job - 100% remote - Proxify AB - /remote-jobs/proxify-ab-senior-python-engineer-long-term-job-100-remote
Full Stack Software Engineer (React / Python) - Trustworthy - /remote-jobs/trustworthy-full-stack-software-engineer-react-python
Senior Open Source Developer & DevOps (Python, Django, React, AWS/OpenStack) - OpenCraft - /remote-jobs/opencraft-senior-open-source-developer-devops-python-django-react-aws-openstack
Senior Backend Engineer (Python) - TestGorilla - /remote-jobs/testgorilla-senior-backend-engineer-python-10
Python Platform Engineer - Doximity - /remote-jobs/doximity-python-platform-engineer
[{'title': 'AI engineer', 'company': 'Sticker Mule', 'link': 'https://remoteok.com/remote-jobs/remote-ai-engineer-sticker-mule-213158'}, {'title': 'Fullstack developer senior', 'company': 'CloudDevs', 'link': 'https://remoteok.com/remote-jobs/remote-fullstack-developer-senior-clouddevs-207318'}, {'title': 'Senior Backend Python Developer', 'company': 'P

127.0.0.1 - - [25/Mar/2023 23:00:43] "GET /export?keyword=python HTTP/1.1" 200 -


Senior Python Engineer: Long-term job - 100% remote - Proxify AB - /remote-jobs/proxify-ab-senior-python-engineer-long-term-job-100-remote
Full Stack Software Engineer (React / Python) - Trustworthy - /remote-jobs/trustworthy-full-stack-software-engineer-react-python
Senior Open Source Developer & DevOps (Python, Django, React, AWS/OpenStack) - OpenCraft - /remote-jobs/opencraft-senior-open-source-developer-devops-python-django-react-aws-openstack
Senior Backend Engineer (Python) - TestGorilla - /remote-jobs/testgorilla-senior-backend-engineer-python-10
Python Platform Engineer - Doximity - /remote-jobs/doximity-python-platform-engineer
